## 사업 보고서 크롤링

1. 사업보고서와 기재 정정된 사업보고서 -> 같은 날짜 기준 최신 것을 가져옴
2. 보고서마다 다른 형식 -> 이사의 경영진단 및 분석만 가져옴

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import webbrowser
import requests
import re

1. API key로 url에 접근 : 하루 만개의 할당량 존재
2. 회사(종목)코드는 상장법인목록에서 가져옴
출처: http://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage
3. 기준 날짜 20140101 이후 오늘(20190117)까지

In [2]:
auth_key="a2f0104fc81d7dc05f8817768aae6c55169516a6" #authority key
company_codes = pd.read_excel('상장법인목록.xlsx',converters={'종목코드':str})
company_codes1 = company_codes.loc[:,"종목코드"]
start_date="20140101"
company_codes.head(2)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,엘에이티,311060,특수 목적용 기계 제조업,"In-Line Sputter, OLED 패널 청정물류 이송 장비 등",2018-12-28,12월,박강일,http://www.latplasma.com,경기도
1,대원모방,311840,직물직조 및 직물제품 제조업,"섬유, 모방",2018-12-27,12월,김석만,NaN,충청북도


In [3]:
# 접수번호(rcp_no)에 해당하는 모든 하위 보고서 URL을 추출하여 리스트로 반환하는 함수
def get_sub_report_urls(rcp_no):
    doc_urls = []
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)
    reg = re.compile('viewDoc\((.*)\);')
    params = []
    matches = reg.findall(r.text)
    for m in matches: 
        params.append(m.replace("'", "").replace(" ", "").split(","))
   
    doc_url_tmpl = "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 

    for p in params:
        if rcp_no == p[0]:
            doc_urls.append( doc_url_tmpl % tuple(p) )
        
    return doc_urls

In [4]:
# 해당 url에서 텍스트를 가져오는 함수
# 일반 사업보고서에는 해당 사업보고서의 url에서 18번째 프레임에 '이사의 경영진단 및 분석의견' 존재
# 그렇지 않으면 앞뒤로 더 찾아서 '이사의 경영진단 및 분석의견' 을 포함한 텍스트의 인덱스 찾기
def get_text_urls(rcp_no) :
    url = get_sub_report_urls(rcp_no)
    le = len(url)
    if le <= 18: idx = 1
    else: idx = 18
    while True:
        url2 = url[idx]
        r = requests.get(url2)
        html_text = r.text
        # 이사의 경영진단 및 분석의견이 아닌 글은 다시 찾기
        if "V. 이사의 경영진단 및 분석의견" not in html_text: idx += 1 
        else : break
        if idx >= le : idx = 0
    return html_text

### TEST
상장법인목록에서 찾고 싶은 회사 코드를 찾아서 테스트 해보자.

In [5]:
#'줌인터넷'으로 일단 해보자
com_idx=41
url = "http://dart.fss.or.kr/api/search.xml?auth="+auth_key+"&crp_cd="+company_codes1[com_idx]+"&start_dt="+start_date+"&bsn_tp=A001"
com_name = company_codes["회사명"][com_idx]
resultXML=urlopen(url)  #this is for response of XML
result=resultXML.read() #Using read method
xmlsoup=BeautifulSoup(result,'html.parser')

data = pd.DataFrame()
te=xmlsoup.findAll("list")
tmp = ''
for t in te:
    if tmp == t.rpt_nm.string.split()[1]: continue
    temp=pd.DataFrame(([[t.crp_cls.string,t.crp_nm.string,t.crp_cd.string,t.rpt_nm.string,
                         t.rcp_no.string,t.flr_nm.string,t.rcp_dt.string, t.rmk.string]]),
        columns=["crp_cls","crp_nm","crp_cd","rpt_nm","rcp_no","flr_nm","rcp_dt","rmk"])
    data=pd.concat([data,temp])
    tmp = t.rpt_nm.string.split()[1]
    print(tmp) # 사업보고서 제목에 나타난 날짜(날짜가 같으면 최근 보고서 우선)

data=data.reset_index(drop=True)
rcp_no = data["rcp_no"]
print(url) # 회사의 사업보고서 리스트를 담은 url
print(rcp_no) # 접수번호
data

URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>

In [71]:
txt = pd.DataFrame(columns = ["회사명","접수번호","텍스트"])
rcp_idx = rcp_no[4] #'20140331000448

url = get_sub_report_urls(rcp_idx)
le = len(url)
idx = 18
while True:
    url2 = url[idx]
    r = requests.get(url2)
    html_text = r.text
    # 이사의 경영진단 및 분석의견이 아닌 글은 다시 찾기
    if "V. 이사의 경영진단 및 분석의견" not in html_text: idx += 1
    else : break
    if idx >= le: idx = 0

txt = txt.append(pd.Series([com_name,rcp_idx,html_text], index = ["회사명","접수번호","텍스트"]) , ignore_index = True)
txt.head()

,회사명,접수번호,텍스트
0,아주IB투자,20140331000448,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."


In [73]:
txt = pd.DataFrame(columns = ["회사명","접수번호","텍스트"])
for rcp_idx in rcp_no:
    html_txt = get_text_urls(rcp_idx)
    txt = txt.append(pd.Series([com_name,rcp_idx,html_txt], index = ["회사명","접수번호","텍스트"]) , ignore_index = True)

In [74]:
txt

,회사명,접수번호,텍스트
0,아주IB투자,20180330003925,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
1,아주IB투자,20170331001194,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
2,아주IB투자,20160330002097,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
3,아주IB투자,20150331004178,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
4,아주IB투자,20140331000448,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."


In [91]:
len(company_codes)

2263

In [ ]:
##  프로세스바
import sys
def progressBar(value, endvalue, bar_length=20):
    percent = float(value) / endvalue
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write("\rPercent: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()

100개만 해보자

In [ ]:
######################### 상장법인목록에 있는 기업들의 사업보고서 크롤링
start = datetime.datetime.now()

txt = pd.DataFrame(columns = ["회사명","회사코드","보고서 명","접수번호","접수날짜","텍스트"])
for com_idx in range(0, 100) :  ## 10개의 기업만 보고 싶은 경우 len(company_codes)를 10 으로 수정
    url = "http://dart.fss.or.kr/api/search.xml?auth="+auth_key+"&crp_cd="+company_codes1[com_idx]+"&start_dt="+start_date+"&bsn_tp=A001"
    com_name = company_codes["회사명"][com_idx]
    resultXML=urlopen(url)  #this is for response of XML
    result=resultXML.read() #Using read method
    xmlsoup=BeautifulSoup(result,'html.parser')
    
    data = pd.DataFrame()
    te=xmlsoup.findAll("list")
    if len(te) == 0 : continue
    tmp = ''
    for t in te:
        if tmp == t.rpt_nm.string.split()[1]: continue
        temp=pd.DataFrame(([[t.crp_cls.string,t.crp_nm.string,t.crp_cd.string,t.rpt_nm.string,
                             t.rcp_no.string,t.flr_nm.string,t.rcp_dt.string, t.rmk.string]]),
            columns=["crp_cls","crp_nm","crp_cd","rpt_nm","rcp_no","flr_nm","rcp_dt","rmk"])
        data=pd.concat([data,temp])
        tmp = t.rpt_nm.string.split()[1]
        
    data = data.reset_index(drop=True)
    
    #엑셀 파일로 저장하기 위해 변수들 저장
    for idx in range(0, len(data)):
        rcp_idx = data["rcp_no"][idx]
        crp_code = data["crp_cd"][idx]
        rpt_nm = data["rpt_nm"][idx]
        rcp_dt = data["rcp_dt"][idx]
        if len(get_sub_report_urls(rcp_idx))<18 : continue
        html_txt = get_text_urls(rcp_idx)
        txt2 = pd.Series([com_name, crp_code, rpt_nm, rcp_idx, rcp_dt, html_txt], 
                         index = ["회사명","회사코드","보고서 명","접수번호","접수날짜","텍스트"])
        txt = txt.append(txt2 , ignore_index = True)

    progressBar(len(txt),len(company_codes))
    
end = datetime.datetime.now()
print("\n걸린시간 : ",end-start)

In [ ]:
txt.head()

### 전체

In [95]:
######################### 상장법인목록에 있는 기업들의 사업보고서 크롤링 ( 오 래 걸 림 ) ###################################
start = datetime.datetime.now()

txt = pd.DataFrame(columns = ["회사명","회사코드","보고서 명","접수번호","접수날짜","텍스트"])
for com_idx in range(0, len(company_codes)) :  ## 10개의 기업만 보고 싶은 경우 len(company_codes)를 10 으로 수정
    url = "http://dart.fss.or.kr/api/search.xml?auth="+auth_key+"&crp_cd="+company_codes1[com_idx]+"&start_dt="+start_date+"&bsn_tp=A001"
    com_name = company_codes["회사명"][com_idx]
    resultXML=urlopen(url)  #this is for response of XML
    result=resultXML.read() #Using read method
    xmlsoup=BeautifulSoup(result,'html.parser')
    
    data = pd.DataFrame()
    te=xmlsoup.findAll("list")
    if len(te) == 0 : continue
    tmp = ''
    for t in te:
        if tmp == t.rpt_nm.string.split()[1]: continue
        temp=pd.DataFrame(([[t.crp_cls.string,t.crp_nm.string,t.crp_cd.string,t.rpt_nm.string,
                             t.rcp_no.string,t.flr_nm.string,t.rcp_dt.string, t.rmk.string]]),
            columns=["crp_cls","crp_nm","crp_cd","rpt_nm","rcp_no","flr_nm","rcp_dt","rmk"])
        data=pd.concat([data,temp])
        tmp = t.rpt_nm.string.split()[1]
        
    data = data.reset_index(drop=True)
    
    #엑셀 파일로 저장하기 위해 변수들 저장
    for idx in range(0, len(data)):
        rcp_idx = data["rcp_no"][idx]
        crp_code = data["crp_cd"][idx]
        rpt_nm = data["rpt_nm"][idx]
        rcp_dt = data["rcp_dt"][idx]
        if len(get_sub_report_urls(rcp_idx))<18 : continue
        html_txt = get_text_urls(rcp_idx)
        txt2 = pd.Series([com_name, crp_code, rpt_nm, rcp_idx, rcp_dt, html_txt], 
                         index = ["회사명","회사코드","보고서 명","접수번호","접수날짜","텍스트"])
        txt = txt.append(txt2 , ignore_index = True)

    progressBar(len(txt),len(company_codes))
    
end = datetime.datetime.now()
print("\n걸린시간 : ",end-start)

ConnectionError: HTTPConnectionPool(host='dart.fss.or.kr', port=80): Max retries exceeded with url: /report/viewer.do?rcpNo=20140331001180&dcmNo=4108523&eleId=1&offset=659&length=4049&dtd=dart3.xsd (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C4B13E2D68>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [93]:
txt.head()

,회사명,회사코드,보고서 명,접수번호,접수날짜,텍스트
0,엘에이티,311060,사업보고서 (2017.12),20190102000129,20190102,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
1,대원모방,311840,사업보고서 (2017.12),20181228000183,20181228,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
2,지놈앤컴퍼니,314130,사업보고서 (2017.12),20181231000061,20181231,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
3,테크엔,308700,사업보고서 (2017.12),20181224000149,20181224,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
4,프로테옴텍,303360,[기재정정]사업보고서 (2017.12),20181226000186,20181226,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."


In [94]:
txt.tail()

,회사명,회사코드,보고서 명,접수번호,접수날짜,텍스트
3263,KB금융,105560,사업보고서 (2013.12),20140331002917,20140331,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
3264,세운메디칼,100700,사업보고서 (2017.12),20180402003912,20180402,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
3265,세운메디칼,100700,사업보고서 (2016.12),20170331001933,20170331,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
3266,세운메디칼,100700,[기재정정]사업보고서 (2015.12),20160331000324,20160331,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."
3267,세운메디칼,100700,[기재정정]사업보고서 (2014.12),20150331003923,20150331,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T..."


In [ ]:
writer = pd.ExcelWriter('./dart_text2.xlsx')
data.to_excel(writer,'Sheet1')
writer.save()